# Байесовская теория классификации

## Библиотеки

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm
from scipy.spatial.distance import cdist
from scipy import stats
import numpy as np
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Принцип максимума правдоподобия

Если $P\bigr(y\bigr)$ равномерная случайная величина, то
$$
a\bigr(x\bigr) = \arg\max_{y\in Y} p\bigr(x|y\bigr)
$$

In [ ]:
p_x_1 = stats.multivariate_normal([-1, -1], np.eye(2))
p_x_2 = stats.multivariate_normal([1, 1], 1.5*np.eye(2))

In [ ]:
def plot():
    x = np.linspace(-3, 3, 300)
    y = np.linspace(-3, 3, 300)
    xs, ys = np.meshgrid(x, y)
    scores = [np.zeros_like(xs), np.zeros_like(xs)]
    for i in range(len(xs)):
        for j in range(len(xs[i])):
            scores[0][i][j] = p_x_1.pdf([xs[i][j],ys[i][j]])
            scores[1][i][j] = p_x_2.pdf([xs[i][j],ys[i][j]])

    ax = plt.figure(figsize=(14.0, 6.0)).gca(projection='3d')
    alpha=0.7
    ax.plot_surface(xs, ys, 
                    np.where(scores[1] <= scores[0], scores[1], np.nan),
                    linewidth=0, color='blue', alpha=alpha)
    ax.plot_surface(xs, ys, 
                    np.where(scores[0] <= scores[1], scores[0], np.nan), 
                    linewidth=0, color='red', alpha=alpha)
    ax.plot_surface(xs, ys, 
                    np.where(scores[1] >= scores[0], scores[1], np.nan),
                    linewidth=0, color='blue', alpha=alpha)
    ax.plot_surface(xs, ys, 
                    np.where(scores[0] >= scores[1], scores[0], np.nan), 
                    linewidth=0, color='red', alpha=alpha)
    ax.set_xlabel(r'x1', labelpad= 14), ax.set_ylabel(r'x2', labelpad=14)
    ax.set_zlabel(r'p(x)'), ax.view_init(30, -120)
    ax.view_init(30, -70)
    plt.show()

plot()

В случае, если плотности $p(x|y)$ заданы, то задача классификации является решенной. Но вопрос как найти $p(x|y)$?

## Одномерный случай

### Синтетические данные

Рассмотрим две гаусианы с разными средними и дисперсиями. Сгенерим выборку, где каждая гаусиана описывает свой класс.

In [ ]:
np.random.seed(42)

l = 10000
p_x_1 = stats.norm(-1, 0.5)
x_1 = np.sqrt(0.5)*np.random.randn(l)-1
p_x_2 = stats.norm(1, 1.5)
x_2 = np.sqrt(1.5)*np.random.randn(l)+1

In [ ]:
x = np.linspace(min(np.min(x_1), np.min(x_2)), 
                max(np.max(x_1), np.max(x_2)), 100)

plt.plot(x_1, x_1*0, '.', color='blue')
plt.show()

In [ ]:
x = np.linspace(min(np.min(x_1), np.min(x_2)), 
                max(np.max(x_1), np.max(x_2)), 100)

plt.plot(x_2, x_2*0, '.', color='red')
plt.show()

In [ ]:
x = np.linspace(min(np.min(x_1), np.min(x_2)), max(np.max(x_1), np.max(x_2)), 100)
plt.plot(x, p_x_1.pdf(x), color='blue', label='class 1')
plt.plot(x, p_x_2.pdf(x), color='red', label='class 2')
plt.legend(loc='best')
plt.show()

### Востановления плотности по эмпирическим данным
Истинное распределение $p(x|y)$ не известно, востановим данную плотность $\hat{p}(x|y, \mathbf{X})$.

In [ ]:
plt.hist(x_1, bins=100)
plt.hist(x_2, bins=100)

plt.show()

In [ ]:
def p(x, D, h = 0.2):
    D = np.array(D)
    x = np.array(x)
    l = len(D)
    n = 1
    if len(D.shape) == 2:
        n = D.shape[1]
        
    D = D.reshape([-1, n])
    x = x.reshape([-1, n])
    return (1/(2*h))*(1/len(D))*(cdist(D, x, metric='minkowski', p=1) <= h).sum(axis=0)


In [ ]:
x = np.linspace(min(np.min(x_1), np.min(x_2)), max(np.max(x_1), np.max(x_2)), 100)

plt.plot(x, p(x, x_1), color='blue', label='p(x|1)')
plt.plot(x, p(x, x_2), color='red', label='p(x|2)')
plt.legend(loc='best')
plt.show()

In [ ]:
x = np.linspace(min(np.min(x_1), np.min(x_2)), 
                max(np.max(x_1), np.max(x_2)), 500)

p1 = p(x, x_1)
p2 = p(x, x_2)
plt.plot(x, p1, color='blue', label='p(x|1)')
plt.plot(x, p2, color='red', label='p(x|2)')

idx = np.argwhere(np.diff(np.sign(p1 - p2))).flatten()
plt.plot(x[idx], p1[idx], 'ko', label='threshold')

plt.legend(loc='best')
plt.show()

x[idx]

### LOO для выбора ширины окна

In [ ]:
def LOO(D, h):
    D_list = D.tolist()
    
    for i in range(len(D_list)):
        return -np.log(p([D_list[i]], D_list[:i] + D_list[i+1:], h)).sum()

In [ ]:
hs = np.linspace(1e-10, 2, 1000)
scores_1 = [LOO(x_1, h) for h in hs]
scores_2 = [LOO(x_2, h) for h in hs]

plt.plot(hs, scores_1, color='blue', label='h for class 1')
plt.plot(hs, scores_2, color='red', label='h for class 2')

plt.legend(loc='best')
plt.show()

In [ ]:
hs[np.argmin(scores_1)], hs[np.argmin(scores_2)]

In [ ]:
x = np.linspace(min(np.min(x_1), np.min(x_2)), 
                max(np.max(x_1), np.max(x_2)), 500)

p1 = p(x, x_1, hs[np.argmin(scores_1)])
p2 = p(x, x_2, hs[np.argmin(scores_2)])
plt.plot(x, p1, color='blue', label='p(x|1)')
plt.plot(x, p2, color='red', label='p(x|2)')

idx = np.argwhere(np.diff(np.sign(p1 - p2))).flatten()
plt.plot(x[idx], p1[idx], 'ko', label='threshold')

plt.legend(loc='best')
plt.show()

x[idx]

## Двумерный случай

In [ ]:
np.random.seed(42)

l = 10000
p_x_1 = stats.multivariate_normal([-1, -1], 0.5*np.eye(2))
x_1 = np.sqrt(0.5)*np.random.randn(l, 2)+np.array([-1, -1])

p_x_2 = stats.multivariate_normal([1, 1], 1.5*np.eye(2))
x_2 = np.sqrt(1.5)*np.random.randn(l, 2)+np.array([1, 1])

In [ ]:
def plot():
    x = np.linspace(-3, 3, 300)
    y = np.linspace(-3, 3, 300)
    xs, ys = np.meshgrid(x, y)
    scores = [np.zeros_like(xs), np.zeros_like(xs)]
    for i in range(len(xs)):
        for j in range(len(xs[i])):
            scores[0][i][j] = p_x_1.pdf([xs[i][j],ys[i][j]])
            scores[1][i][j] = p_x_2.pdf([xs[i][j],ys[i][j]])

    ax = plt.figure(figsize=(14.0, 6.0)).gca(projection='3d')
    alpha=0.7
    ax.plot_surface(xs, ys, 
                    np.where(scores[1] <= scores[0], scores[1], np.nan),
                    linewidth=0, color='blue', alpha=alpha)
    ax.plot_surface(xs, ys, 
                    np.where(scores[0] <= scores[1], scores[0], np.nan), 
                    linewidth=0, color='red', alpha=alpha)
    ax.plot_surface(xs, ys, 
                    np.where(scores[1] >= scores[0], scores[1], np.nan),
                    linewidth=0, color='blue', alpha=alpha)
    ax.plot_surface(xs, ys, 
                    np.where(scores[0] >= scores[1], scores[0], np.nan), 
                    linewidth=0, color='red', alpha=alpha)
    ax.set_xlabel(r'x1', labelpad= 14), ax.set_ylabel(r'x2', labelpad=14)
    ax.set_zlabel(r'p(x)'), ax.view_init(30, -120)
    ax.view_init(30, -70)
    plt.show()

plot()

In [ ]:
hs = np.linspace(1e-10, 5, 20)
scores_1 = [LOO(x_1, h) for h in hs]
scores_2 = [LOO(x_2, h) for h in hs]

plt.plot(hs, scores_1, color='blue', label='h for class 1')
plt.plot(hs, scores_2, color='red', label='h for class 2')

plt.legend(loc='best')
plt.show()

In [ ]:
hs[np.argmin(scores_1)], hs[np.argmin(scores_2)]

In [ ]:
def plot():
    x = np.linspace(-3, 3, 300)
    y = np.linspace(-3, 3, 300)
    xs, ys = np.meshgrid(x, y)
    scores = [np.zeros_like(xs), np.zeros_like(xs)]
    for i in range(len(xs)):
        line = np.array([[xs[i][j],ys[i][j]] for j in range(len(xs[i]))])
        scores[0][i] = p(line, x_1, hs[np.argmin(scores_1)])
        scores[1][i] = p(line, x_2, hs[np.argmin(scores_2)])

    ax = plt.figure(figsize=(14.0, 6.0)).gca(projection='3d')
    alpha=0.7
    ax.plot_surface(xs, ys, 
                    np.where(scores[1] <= scores[0], scores[1], np.nan),
                    linewidth=0, color='blue', alpha=alpha)
    ax.plot_surface(xs, ys, 
                    np.where(scores[0] <= scores[1], scores[0], np.nan), 
                    linewidth=0, color='red', alpha=alpha)
    ax.plot_surface(xs, ys, 
                    np.where(scores[1] >= scores[0], scores[1], np.nan),
                    linewidth=0, color='blue', alpha=alpha)
    ax.plot_surface(xs, ys, 
                    np.where(scores[0] >= scores[1], scores[0], np.nan), 
                    linewidth=0, color='red', alpha=alpha)
    ax.set_xlabel(r'x1', labelpad= 14), ax.set_ylabel(r'x2', labelpad=14)
    ax.set_zlabel(r'p(x)'), ax.view_init(30, -120)
    ax.view_init(30, -70)
    plt.show()

plot()

## Параметрическое востановление плотности

### Принцип максимума правдоподобия

$$
\hat{\theta} = \arg\max_{\theta}L\bigr(\theta, \mathbf{X}^{m}\bigr) = \sum_{i=1}^{m}\ln p\bigr(x_i| \theta\bigr)
$$

Пример для нормального распределения:
$$
\mu = \frac{1}{m}\sum_{i=1}^{m}x_i.
$$

## Нормировка данных

Для реальных данных всегда нужно проводить предварительный анализ данных. В частности требуется выполнить нормировку данных. Нормировка данных позволяет повысить устойчивость модели при обучении.

In [ ]:
X, Y = make_classification(n_samples=150, n_features=2, 
                           n_informative=2, n_classes=2, 
                           n_redundant=0,
                           n_clusters_per_class=1,
                           random_state=40)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=50, 
                                                    random_state=0)

In [ ]:
model = SVC(kernel='linear')
_ = model.fit(X_train, Y_train)
model.score(X_test, Y_test)

In [ ]:
fig, gs = plt.figure(figsize=(14,4)), gridspec.GridSpec(1, 3)

ax = []
for i in range(3):
    ax.append(fig.add_subplot(gs[i]))


plot_decision_regions(X_train, Y_train, model, ax=ax[0])
plot_decision_regions(X_test, Y_test, model, ax=ax[1])
plot_decision_regions(X_train[model.support_], Y_train[model.support_], model, ax=ax[2])

plt.show()

In [ ]:
np.random.seed(0)
X, Y = make_classification(n_samples=150, n_features=2, 
                           n_informative=2, n_classes=2, 
                           n_redundant=0,
                           n_clusters_per_class=1,
                           random_state=40)

X = (X + 1000*np.random.randn(1, 2)) * 1000*np.random.randn(1, 2)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=50, random_state=0)

In [ ]:
model = SVC(kernel='linear')
_ = model.fit(X_train, Y_train)
model.score(X_test, Y_test)

In [ ]:
fig, gs = plt.figure(figsize=(14,4)), gridspec.GridSpec(1, 3)

ax = []
for i in range(3):
    ax.append(fig.add_subplot(gs[i]))


plot_decision_regions(X_train, Y_train, model, ax=ax[0])
plot_decision_regions(X_test, Y_test, model, ax=ax[1])
plot_decision_regions(X_train[model.support_], Y_train[model.support_], model, ax=ax[2])

plt.show()

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = SVC(kernel='linear')
_ = model.fit(X_train, Y_train)
model.score(X_test, Y_test)

In [ ]:
fig, gs = plt.figure(figsize=(14,4)), gridspec.GridSpec(1, 3)

ax = []
for i in range(3):
    ax.append(fig.add_subplot(gs[i]))


plot_decision_regions(X_train, Y_train, model, ax=ax[0])
plot_decision_regions(X_test, Y_test, model, ax=ax[1])
plot_decision_regions(X_train[model.support_], Y_train[model.support_], model, ax=ax[2])

plt.show()

# Ирисы Фишера
Выборка взята отсюда: https://archive.ics.uci.edu/ml/datasets/iris

## Загрузка выборки

In [ ]:
dataset = pd.read_csv('data/iris.csv', 
                      header=None, 
                      names=['длина чашелистика', 'ширина чашелистика', 
                             'длина лепестка', 'ширина лепестка', 'класс'])
dataset.sample(5, random_state=0)

## Начало работы с данными
1. Определить множество объектов:
    * Определить размер выборки
    * Определить признаки, которыми описываются объекты
2. Определить множество ответов
3. Определить тип задачи машинного обучения
6. ...

### Множество объектов
В данной задачи множество объектов описывается $n=4$ признаками:
1. Длина чашелистика
2. Ширина чашелистика
3. Длина лепестка
4. Ширина лепестка

In [ ]:
print('Размер выборки составляет l={} объектов.'.format(len(dataset)))

Все признаки являются вещественными признаками. Формально объекты $\mathbf{X}$ представляються в следующем виде:
$$\mathbf{X} \in \mathbb{R}^{l\times n},$$
где $l$ число объектов, а $n$ число признаков.

Получаем, что $\mathbf{X}$ это некоторая вещественная матрица размера $l\times n$.

### Множество ответов
В данной задаче множество ответов состоит из трех элементов:
1. Iris-virginica
2. Iris-versicolor
3. Iris-setosa

### Задача машинного обучения
В нашем случае, так как мощность множества $|\mathbf{y}|=3 \ll l=150$ получаем задачу классификации на $M=3$ класса.

## Анализ данных
Сначала проэктируем все объекты на двумерные плоскости, для упрощения анализа

In [ ]:
sns.pairplot(dataset, hue='класс', height=2)
plt.show()

Из рисунка видно, что класс синих точек (Iris-setosa) легко отделяется от двух других цветов. Оранжевые и зеленные точки отделяются не так просто в каждой из проэкций, но все равно можно провести прямую, которая отделит оранджевые точки от зеленых.

## Построение модели
### Преобразование данных
Как было сказано ранее нам требуется решить задачу классификации на 3 класса. Но для наглядноси рассмотрим бинарную классификацию (классификацию на несколько классов рассмотрим в следующей лекции).

Чтобы исходную задачу преобразовать в задачу бинарной классификации уберем из выборки все объекта класса Iris-setosa.

In [ ]:
binary_dataset = dataset.drop(index=dataset.index[dataset['класс'] == 'Iris-setosa'])

Классы закодируем целыми числами $-1$ и $1$.

In [ ]:
binary_dataset.loc[dataset['класс'] == 'Iris-versicolor', dataset.columns == 'класс'] = -1
binary_dataset.loc[dataset['класс'] == 'Iris-virginica', dataset.columns == 'класс'] = 1

Получаем задачу бинарной классификации.

### Модель алгоритмов

Модель алгоритмов $\mathfrak{F}$ в машинном обучении это некоторое множество функций, которые действуют из множества объектов в множество ответов, в нашем случае:
$$\mathfrak{F} = \{f| f: \mathbb{R}^n \to \{-1, 1\}, \text{еще какие-то ограничения}\},$$
обычно $\mathfrak{F}$ это некоторое параметрическое семество функций, тоесть разные функции $f$ отличаются друг от друга только каким-то параметром. Простым примером параметрическим семейством функций для задачи бинарной классификации является семейство линейный классификатор:
$$\mathfrak{F}_{bcl} = \left\{f\bigr(\theta, \mathbf{x}\bigr)=\text{sign}\bigr(\theta^{\mathsf{T}}\mathbf{x}\bigr)\bigr| \theta \in \mathbb{R}^{n} \right\}.$$


### Функция потерь

Машиное обучение это всегда выбор функции из множества $\mathfrak{F}$. Чтобы выбрать функцию, нужен некоторый критерий по которому она выбирается, то есть нужно упоррядочить все функции от худшей к лучшей. Для этого построем функционал $\mathcal{L}$, который каждой функции $f \in \mathfrak{F}$ ставит в соответствии число из $\mathbb{R}_+$. В машинном обучении обычно функционал качества водиться как некоторая ошибка на выборке. В общем виде функционал качества можно представить в следующем виде:
$$\mathcal{L}\bigr(f, \mathbf{X}, \mathbf{y}\bigr) = \sum_{i=1}^l\mathcal{q}\bigr(f, \mathbf{x}_i, y_i\bigr),$$
где $q$ некоторая функция ошибки на некотором объекте $\mathbf{x}$. Функционал качества $\mathcal{L}$ называется эмперическим риском.

### Оптимизационная задача

Далее нужно поставить задачу оптимизации для выбора $f \in \mathfrak{F}$. Здесь все просто, просто минимизируем эмперический риск:
$$\hat{f} = \arg \min_{f \in \mathfrak{F}} \mathcal{L}\bigr(f, \mathbf{X}, \mathbf{y}\bigr).$$

Важно! В результе функция $\hat{f}$ зависит от выборки $\left(\mathbf{X}, \mathbf{y}\right)$, то есть для разных наборов данных оптимальная функция будет различная.

Вернемся к нашей задаче. В нашем случае функционал качества будет иметь следующий вид:
$$\mathcal{L}\bigr(\theta, \mathbf{X}, \mathbf{y}\bigr) = \sum_{i=1}^l\bigr[f\bigr(\theta, \mathbf{x}_i\bigr) \not= y_i\bigr],$$
и оптимизационная задача переписывается в виде:
$$\hat{\theta} = \arg \min_{\theta \in \mathbb{R}^n} \sum_{i=1}^l\bigr[f\bigr(\theta, \mathbf{x}_i\bigr) \not= y_i\bigr].$$

Воспользуемся библиотеками для решения данной задачи. Далее в примере будет найден параметр $\hat{\theta}$ не как решение непосредственно этой оптимизационной задачи, а немного измененной, но об этом позже в следующей лекции.

### Поиск оптимального вектора параметров
Перейдем к двум матрицам:
1. Матрице объектов $\mathbf{X} \in \mathbb{R}^{l\times (n+1)}$
2. Вектору ответов $\mathbf{y} \in \{-1,1\}^l$

Заметим, что объекты мы погрузили в пространство более большой размерности, добавив еще один признак, который у всех объектов будет равен $1$.

In [ ]:
X = binary_dataset.iloc[:, binary_dataset.columns != 'класс'].values
y = binary_dataset.iloc[:, binary_dataset.columns == 'класс'].values.reshape(-1)
X = np.array(np.hstack([X, np.ones([len(X), 1])]), dtype=np.float64)
y = np.array(y, dtype=np.int64)

In [ ]:
model = LogisticRegression(random_state=0, max_iter=2000)
_ = model.fit(X, y)

Получаем вектор оптимальных параметров $\hat{\theta}$

In [ ]:
model.coef_

# Переход от бинарной классификации к многоклассовой
Теперь же остается вопрос: как перейти от задачи бинарной классификации к многоклассовой? В качестве бинарного классификатора рассмотрим все ту же модель алгоритмов: 
$$\mathfrak{F}_{bcl} = \left\{f\bigr(\theta, \mathbf{x}\bigr)=\text{sign}\bigr(\theta^{\mathsf{T}}\mathbf{x}\bigr)\bigr| \theta \in \mathbb{R}^{n} \right\}.$$

Но знак позволяет отделить только два знака. Какие же есть решения?

Рассмотрим вариант перехода, который называется Один против всех (One VS All). Для простоты визуализации рассмотрим пример на синтетических данных. 

## Генерация синтетической выборки

In [ ]:
np.random.seed(0)
l = 100
n = 2
X1 = np.array([[-1,-1]]) + 0.5*np.random.randn(l, n)
X2 = np.array([[1,1]]) + 0.5*np.random.randn(l, n)
X3 = np.array([[-1,1]]) + 0.5*np.random.randn(l, n)

X = np.vstack([X1, X2, X3])
y = np.hstack([[0]*l, [1]*l, [2]*l])

# Добавляем константу
X = np.hstack([X, np.ones([len(X), 1])])


In [ ]:
cols = ['blue', 'red', 'green']

# построение точек
for k in np.unique(y):
    plt.plot(X[y==k,0], X[y==k,1], 'o', label='класс {}'.format(k), color=cols[k])

plt.legend(loc='best')
plt.show()

## Один против всех

Данные метод основан на том, что для классификации на $M>2$ классов нужно построить $M$ линейных классификаторов, которые классифицируют $k$-й класс прорив всех остальных классов.

Построим $M=3$ классификатора, которые отделяют каждый класс от двух остальных

In [ ]:
models = []
model = LogisticRegression(random_state=0, max_iter=2000, fit_intercept=False)
_ = model.fit(X, np.array(y==0, dtype=np.int64))
models.append(model)

model = LogisticRegression(random_state=0, max_iter=2000, fit_intercept=False)
_ = model.fit(X, np.array(y==1, dtype=np.int64))
models.append(model)

model = LogisticRegression(random_state=0, max_iter=2000, fit_intercept=False)
_ = model.fit(X, np.array(y==2, dtype=np.int64))
models.append(model)

In [ ]:
def get_line(a, b, c=0, x_min=-10, x_max=10):
    x1, y1 = -(-models[k].coef_[0][1] + c)/models[k].coef_[0][0], -1
    x2, y2 = -(models[k].coef_[0][1] + c)/models[k].coef_[0][0], 1
    
    polynomial = np.poly1d(np.polyfit([x1, x2], [y1, y2], 1))
    x_axis = np.linspace(x_min, x_max)
    y_axis = polynomial(x_axis)
    
    return x_axis, y_axis

In [ ]:
cols = ['blue', 'red', 'green']
plt.xlim((-2.5, 2.5))
plt.ylim((-2.5, 2.5))

for k in np.unique(y):
    plt.plot(X[y==k,0], X[y==k,1], 'o', 
             label='класс {}'.format(k), color=cols[k])

for k in np.unique(y):
    plt.plot(*get_line(*models[k].coef_[0]), linewidth=2, color=cols[k])
    print(models[k].coef_[0])

plt.legend(loc='best')
plt.show()

## Методы анализа качества

Самый простой способо это подсчет ошибок, не верных классов:
$$\mathcal{L}\bigr(\theta, \mathbf{X}, \mathbf{y}\bigr) = \sum_{i=1}^l\bigr[f\bigr(\theta, \mathbf{x}_i\bigr) \not= y_i\bigr].$$

Также можно рассмотреть таблицу попарных ошибок:

|               | y = 1 | y = 2 | y = 3 |
| ------------- | ----- | ----- | ----- |
| __f(x) = 1__  |  1-1  |  1-2  |  1-3  |
| __f(x) = 2__  |  2-1  |  2-2  |  2-3  |
| __f(x) = 3__  |  3-1  |  3-2  |  3-3  |

Данная таблица показывает, сколько классификатор сделал ошибок между двумя парами классов. Например:
1. 2-3 это число, которое обозначает, сколько объектов класса $3$ объект отнес к классу $2$
2. 3-1 это число, которое обозначает, сколько объектов класса $1$ объект отнес к классу $3$

Посчитаем данную матрицу для синтетической выборки.

In [ ]:
scores = np.zeros([3,3])
for k in range(3):
    pred = np.argmax(np.vstack([models[i].predict_proba(X[y==k])[:, 1] for i in range(3)]).T, axis=1)
    for i in range(3):
        scores[i, k] = sum(pred == i)
        

In [ ]:
print(scores)

Результаты данной таблицы показывают такой же результат как и картинка. Хуже всего от других отделим второй класс.

## Servo dataset

In [ ]:
numFolds = 10
SEED = 888
np.random.seed(SEED)

colnames = ["motor", "screw", "pgain", "vgain", "class"]
dataset = pd.read_csv("data/servo.data", sep=",", names=colnames)

# Some data preprocessing
X = dataset.drop("class", axis=1)
Y = dataset["class"]
X_conv = pd.get_dummies(X, columns=colnames[:-1])
print(X[0:3])
print(X_conv[0:3])
kf = KFold(numFolds, shuffle=True)

Models = [LinearRegression, SVR]
for Model in Models:
    total = 0
    for train_indices, test_indices in kf.split(X):
        X_train = X_conv.iloc[train_indices, :]
        Y_train = Y[train_indices]
        
        X_test = X_conv.iloc[test_indices, :]
        Y_test = Y[test_indices]

        # Testing out on the linear regression
        reg = Model()
        reg.fit(X_train, Y_train)

        predictions = reg.predict(X_test)
        mse = mean_squared_error(Y_test, predictions)
        #mse = mean_absolute_error(Y_test, predictions)
        total += mse
    
    mse = total / numFolds
    print("Average mse of {0}: {1}", Model.__name__, mse)
